In [1]:
%pip install ee
%pip install geemap

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for ee: filename=ee-0.2-py3-none-any.whl size=3667 sha256=ccbded4cdb0c9f2e00158e8afbff4c0edf69bb7e8d2d9841bcae3c2579244459
  Stored in directory: c:\users\chris\appdata\local\pip\cache\wheels\68\e5\fb\285c24eaa7ce9d3844e311635dea798b3c4c9ceb6c32cd1da4
Successfully built ee
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress t


     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ---------------------------------------- 0.0/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------ --------------------------------- 10.2/60.8 kB ? eta -:--:--
     ------------------- ------------------- 30.7/60.8 kB 87.5 kB/s eta 0:00:01
     ------------------- ------------------- 30.7/60.8 kB 87.5 kB/s eta 0:00:01
     ------------------------------- ------ 51.2/60.8 kB 119.1 kB/s eta 0:00:01
     -------------------------------------- 60.8/60.8 kB 147.1 kB/s eta 0:00:00
  Using cached pyperclip-1

In [3]:
%pip install setuptools

  Using cached setuptools-75.3.0-py3-none-any.whl.metadata (6.9 kB)
Using cached setuptools-75.3.0-py3-none-any.whl (1.3 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import ee
import geemap.foliumap as geemap
import pandas as pd

ee.Authenticate()

# Initialize the Earth Engine API
project_id = 'ee-christianguidikov'  # Replace with your actual project ID
ee.Initialize(project=project_id)

df = pd.read_csv('locust_data.csv')

start_date = '2020-01-01'
end_date = '2020-12-31'

points = []

filtered_df = df[(df['Observation Date'] >= start_date) & (df['Observation Date'] <= end_date) & (df['Category'] == 'Swarm') & (df['Country'] == 'Somalia')]

lon_lat_df = filtered_df[['lat', 'lon']]

lat_long_list = list(zip(lon_lat_df['lat'], lon_lat_df['lon']))

for point in lat_long_list:
    points.append(ee.Geometry.Point(point[1], point[0]).buffer(5000))

    
# Define a region of interest (ROI) centered above Somalia
roi = ee.Geometry.Rectangle([40.9866, -1.6833, 51.1339, 12.1889])

# Load the MODIS NDVI Image Collection
ndvi_collection = ee.ImageCollection("MODIS/061/MOD13A2")
filtered_ndvi = ndvi_collection.filterDate(start_date, end_date).filterBounds(roi)
ndvi_band = filtered_ndvi.select('NDVI').mean().clip(roi)  # Clip to ROI

# Load the SMAP Soil Moisture Image Collection
soil_moisture_collection = ee.ImageCollection("NASA/SMAP/SPL3SMP_E/005")
filtered_soil_moisture = soil_moisture_collection.filterDate(start_date, end_date).filterBounds(roi)
soil_moisture_band = filtered_soil_moisture.select('soil_moisture_am').mean().clip(roi)  # Clip to ROI

longitude = 46.022222  # Replace with your longitude
latitude = 5.974496   # Replace with your latitude
point = ee.Geometry.Point([longitude, latitude]).buffer(50000)

points.append(point)

ndvi_value = ndvi_band.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=point,
    scale=500
).get('NDVI').getInfo()

soil_moisture_value = soil_moisture_band.reduceRegion(
    reducer=ee.Reducer.mean(),
    geometry=point,
    scale=500
).get('soil_moisture_am').getInfo()

# Print the results
print('NDVI Value:', ndvi_value)
print('Soil Moisture Value:', soil_moisture_value)

# Combine the polygons into a feature collection
polygons = ee.FeatureCollection([ee.Feature(p) for p in points])

# Define the style for the polygons
polygon_style = {
    'color': 'red',
    'fillColor': 'red',
}

# Visualize the soil moisture and NDVI data
Map = geemap.Map()
Map.centerObject(roi, 8)
Map.addLayer(soil_moisture_band, {'min': 0, 'max': 0.5, 'palette': ['blue', 'white', 'brown'], 'opacity': 0.5}, 'Mean Soil Moisture')
Map.addLayer(ndvi_band, {'min': 0, 'max': 9000, 'palette': ['white', 'green'], 'opacity': 0.5}, 'Mean NDVI')
Map.addLayer(polygons.style(**polygon_style), {}, 'Large Red Dots')
Map.addLayerControl()
Map


NDVI Value: 2611.7857783746226
Soil Moisture Value: 0.08452596394770154
